## Import library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from matplotlib import font_manager, rc
%matplotlib inline

import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)


# 배경색 흰색으로 맞춰줌
import matplotlib as mpl
mpl.rc('figure', facecolor = 'w', edgecolor ='w')

In [2]:
# Data Load
demo = pd.read_csv('../data/LPOINT_BIG_COMP_01_DEMO.csv', low_memory=False)
pdde = pd.read_csv('../data/LPOINT_BIG_COMP_02_PDDE.csv', low_memory=False)
cop = pd.read_csv('../data/LPOINT_BIG_COMP_03_COP_U.csv', low_memory=False)
clac = pd.read_csv('../data/LPOINT_BIG_COMP_04_PD_CLAC.csv', low_memory=False)
br = pd.read_csv('../data/LPOINT_BIG_COMP_05_BR.csv', low_memory=False)
lpay = pd.read_csv('../data/LPOINT_BIG_COMP_06_LPAY.csv', low_memory=False)

# Column명 변경
demo.columns = ['고객번호', '성별', '연령대', '거주지대분류코드']
pdde.columns = ['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '구매시간', '구매금액', '구매수량']
cop.columns = ['고객번호', '영수증번호', '제휴사', '점포코드', '채널구분', '이용일자', '방문일자', '이용시간', '이용금액']
clac.columns = ['상품코드', '소분류명', '대분류명', '중분류명']
br.columns = ['점포코드', '제휴사', '점포대분류코드', '점포중분류코드']
lpay.columns = ['고객번호', '영수증번호', '제휴사', '채널구분', '이용일자', '이용시간', '이용금액']

## Preprocessing

In [3]:
# 중복되는 행 제거 -> 데이터 추출시 고객, 상품 각각 추출하여 중복이 발생한 것으로 판단함
pdde.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

In [4]:
# 결측치 치환 -> na값을 온라인으로 채워줌
pdde['점포코드'] = pdde['점포코드'].fillna('온라인')

In [5]:
# 결측치 치환
cop['점포코드'] = cop['점포코드'].fillna('온라인')

## Data Merge

In [6]:
# 데이터 크기 확인
print('pdde :',pdde.shape)
print('copu :',cop.shape)
print('lpay :',lpay.shape)

pdde : (4144389, 10)
copu : (248304, 9)
lpay : (353184, 7)


In [7]:
# # 칼럼별 상관관계 확인
# # 제휴사 테이블 중에서 방문일자와 이용일자는 상관관계가 상당히 높게 나타남을 확인
# # 제휴사 테이블 내 방문일자 제거
# plt.figure(figsize=(15,15))
# sns.heatmap(data = cop.corr(), annot=True,fmt = '.2f', linewidths=.5, cmap='Blues')

In [8]:
# 데이터를 통합하기 위하여 칼럼 통합
pdde['타입'] = 'Product'
cop['타입'] = 'Affiliate'
lpay['타입'] = 'Lpay'
pdde['방문일자'] = 0
lpay['방문일자'] = 0
cop['구매수량'] = 0
cop['상품코드'] = np.nan
lpay['점포코드'] = np.nan
lpay['상품코드'] = np.nan
lpay['구매수량'] = 0

In [9]:
# 칼럼명을 새롭게 동일하게 통일
cop.columns = ['고객번호', '영수증번호', '제휴사', '점포코드', '채널구분', '구매일자', '방문일자', '구매시간', '구매금액', '타입', '구매수량', '상품코드']
lpay.columns = ['고객번호', '영수증번호', '제휴사', '채널구분', '구매일자', '구매시간', '구매금액', '타입', '방문일자', '점포코드', '상품코드', '구매수량']

# 상품구매데이터를 기준으로 하여 열을 통일
cop = cop[['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '방문일자', '구매시간', '구매금액', '구매수량', '타입']]
lpay = lpay[['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '방문일자', '구매시간', '구매금액', '구매수량', '타입']]

# 데이터 타입 변경
lpay['영수증번호'] = lpay['영수증번호'].astype('object')
lpay['점포코드'] = lpay['점포코드'].astype('object')
cop['상품코드'] = cop['상품코드'].astype('object')
lpay['상품코드'] = lpay['상품코드'].astype('object')
pdde['구매금액'] = pdde['구매금액'].astype('int')
pdde['방문일자'] = pdde['방문일자'].astype('int64')
lpay['방문일자'] = lpay['방문일자'].astype('int64')

In [10]:
# 데이터 concat
df = pd.concat([pdde,cop,lpay])
# 고객 정보 테이븛 통합
df = pd.merge(df, demo, on = ['고객번호'], how = 'left')
# 상품 분류 테이블 통합
df = pd.merge(df, br, on = ['점포코드','제휴사'], how = 'left')
# 점포 정보 테이블 통합
df = pd.merge(df, clac, on = ['상품코드'], how = 'left')

In [11]:
# 결측치 확인
df.isnull().sum()

고객번호             0
영수증번호            0
채널구분             0
제휴사              0
점포코드        353184
상품코드        601488
구매일자             0
구매시간             0
구매금액             0
구매수량             0
타입               0
방문일자             0
성별               0
연령대              0
거주지대분류코드         0
점포대분류코드     729023
점포중분류코드     729023
소분류명        601488
대분류명        601488
중분류명        601488
dtype: int64

In [12]:
print('<엘페이>')
print(lpay.isnull().sum())
print('<제휴사>')
print(cop.isnull().sum())
print('<상품구매>')
print(pdde.isnull().sum())

<엘페이>
고객번호          0
영수증번호         0
채널구분          0
제휴사           0
점포코드     353184
상품코드     353184
구매일자          0
방문일자          0
구매시간          0
구매금액          0
구매수량          0
타입            0
dtype: int64
<제휴사>
고객번호          0
영수증번호         0
채널구분          0
제휴사           0
점포코드          0
상품코드     248304
구매일자          0
방문일자          0
구매시간          0
구매금액          0
구매수량          0
타입            0
dtype: int64
<상품구매>
고객번호     0
영수증번호    0
채널구분     0
제휴사      0
점포코드     0
상품코드     0
구매일자     0
구매시간     0
구매금액     0
구매수량     0
타입       0
방문일자     0
dtype: int64


In [13]:
data = df.copy()

In [14]:
# 데이터 분리
df1 = data[data['타입']== 'Product']
df2 = data[data['타입']== 'Affiliate']
df3 = data[data['타입']== 'Lpay']

In [15]:
df.head(3)

,고객번호,영수증번호,채널구분,제휴사,점포코드,상품코드,구매일자,구매시간,구매금액,구매수량,타입,방문일자,성별,연령대,거주지대분류코드,점포대분류코드,점포중분류코드,소분류명,대분류명,중분류명
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000,1,Product,0,여성,50대,Z17,Z10,Z10042,남성티셔츠,남성의류,남성의류상의
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700,1,Product,0,여성,40대,Z13,Z13,Z13001,기타주방일회용품,주방잡화,주방일회용품
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000,1,Product,0,여성,50대,Z17,Z10,Z10042,남성티셔츠,남성의류,남성의류상의


### 최종 접속 경과일 (최종 구매 경과일)

In [16]:
df['날짜'] = df['구매일자'].astype(str)
df['날짜'] = pd.to_datetime(df['날짜'])
# df['날짜'] = df['날짜'].dt.tz_localize('UTC')

### 최종 접속 경과일 (최종 구매 경과일)

In [18]:
imp = df.sort_values(by=['날짜'], ascending=[True]).groupby(['고객번호', '날짜'])[['채널구분']].count().reset_index().groupby('고객번호')['날짜'].agg([('구매일수', 'count')]).reset_index()
# 딱 하루만 구매한 고객들
oneday_buyer = list(imp[imp['구매일수']==1]['고객번호'])
# 고객별 구매 날짜
imp2 = df.groupby(['고객번호', '날짜'])[['채널구분']].count().drop('채널구분', axis=1).reset_index()
# 최근 구매일자별 최근 상위 2개를 구함
result = imp2.sort_values(by=['고객번호', '날짜']).groupby('고객번호')['날짜'].apply(set).reset_index()
result['최근상위2'] = result['날짜'].apply(lambda x:sorted(list(x))[-2:])
# 상위 두일자의 차이를 구한 후 빼줌
result['최종구매경과일'] = result['최근상위2'].apply(lambda x: abs(x[1] - x[0]) if len(x) == 2 else abs(pd.to_datetime('2021-12-31') - x[0]))
# int로 변환
result['최종구매경과일'] = result['최종구매경과일'].dt.days
# 결과 도출
result = result[['고객번호','최종구매경과일']];result

,고객번호,구매일수
0,M000034966,8
1,M000059535,1
2,M000136117,45


### 평균 구매일 주기

In [23]:
diff_final_buy = df.groupby('고객번호')['날짜'].diff().fillna('0').astype(str)
diff_final_buy = diff_final_buy.apply(lambda x: abs(int(x.split()[0])))
df['diff'] = diff_final_buy

In [24]:
f = df.groupby(['고객번호'])[['diff']].sum().reset_index()
result['구매간격합'] = f['diff']
result['구매일수'] = imp['구매일수']

In [25]:
result['구매접속간격'] = result['구매일수']-1

In [26]:
# 평균구매주기
mean_buy_cycle = [] 

for i in range(result.shape[0]):
    try:
        mean_buy_cycle.append(result['구매간격합'][i]/result['구매접속간격'][i])

    except ZeroDivisionError:
        mean_buy_cycle.append(1)
        continue

len(mean_buy_cycle)

<ipython-input-26-472f402cf1fe>:6: RuntimeWarning: invalid value encountered in longlong_scalars
  mean_buy_cycle.append(result['구매간격합'][i]/result['구매접속간격'][i])


29874

In [27]:
result['평균구매주기'] = mean_buy_cycle
# 딱 하루 구매한 사람들은 평균구매주기를 1로 대체
result['평균구매주기'].fillna(1, inplace=True)

## 이탈 위험 비율
- [최종구매경과일] / [평균구매주기]

In [28]:
result['이탈위험비율'] = result['최종구매경과일']/result['평균구매주기']

## K-Means Clustering

In [29]:
data = result[['고객번호','이탈위험비율']]
data

,고객번호,이탈위험비율
0,M000034966,1.108504
1,M000059535,303.000000
2,M000136117,1.711111
3,M000201112,2.547619
4,M000225114,0.828947
...,...,...
29869,M999708287,1.000000
29870,M999770689,3.724928
29871,M999849895,2.976667
29872,M999926092,1.394495


## END